### Automotive Sales Data Analysis Project

- Analyze automotive sales data to understand car pricing, trends, and market behavior. 
- Use SQL to clean, aggregate, and explore the dataset, answering questions like average selling price by make/model, condition vs. price, regional sales trends, and date-based patterns. 
- Builds practical SQL skills while simulating real industry analysis.

In [ ]:
-- Identify and remove duplicate VIN entries.

DELETE FROM carprices
WHERE vin in
(
    select vin from
    (
        select 
                vin,
                row_number() over(partition by vin order by year) as dup_vin
            from 
                carprices
    ) t
    where dup_vin > 1
)

In [ ]:
-- Find rows with missing values in sellingprice or mmr.

SELECT
    *
FROM
    carprices
WHERE
    sellingprice is null or mmr is null